In [1]:
using JuMP
using GLPK

In [2]:
function JuliaStringToArray(input)
    ques = zeros(Int, 9, 9)
    arr = split(input,"")
    n = 0
    for i in 1:9
        for j in 1:9
            n = n+1
            ques[i, j] = parse(Int64, arr[n])
        end
    end
    return ques
end

JuliaStringToArray (generic function with 1 method)

In [3]:
function FindSolution(init_sol)
    sudoku = Model(GLPK.Optimizer)
    @variable(sudoku, x[i = 1:9, j = 1:9, k = 1:9], Bin)
    for i in 1:9
        for j in 1:9
            @constraint(sudoku, sum(x[i, j, k] for k in 1:9) == 1)
        end
    end
    for ind in 1:9
        for k in 1:9
            @constraint(sudoku, sum(x[ind, j, k] for j in 1:9) == 1)
            @constraint(sudoku, sum(x[i, ind, k] for i in 1:9) == 1)
        end
    end
    for i in 1:3:7
        for j in 1:3:7
            for k in 1:9
                @constraint(
                    sudoku,
                    sum(x[r, c, k] for r in i:(i+2), c in j:(j+2)) == 1
                )
            end
        end
    end
    for i in 1:9
        for j in 1:9
            # If the space isn't empty
            if init_sol[i, j] != 0
                # Then the corresponding variable for that digit and location must
                # be 1.
                fix(x[i, j, init_sol[i, j]], 1; force = true)
            end
        end
    end
    optimize!(sudoku)
    x_val = value.(x)
    sol = ""
    for i in 1:9
        for j in 1:9
            for k in 1:9
                if round(Int, x_val[i, j, k]) == 1
                    sol = sol * string(k)
                end
            end
        end
    end
    return sol
end

FindSolution (generic function with 1 method)

In [4]:
function compareSolution(sol,answer)
    if cmp(sol, answer)==0
        println("Answer is correct")
    else
        println("Answer is wrong")
    end
end

compareSolution (generic function with 1 method)

In [11]:
input = "040800500080760092001005470056309000009001004320500010000200700700090030005008026,947812563583764192261935478156349287879621354324587619698253741712496835435178926"
arr = split(input,',')
ques = arr[1]
answer = arr[2]
solution = FindSolution(JuliaStringToArray(ques))
compareSolution(solution,answer)

Answer is correct


In [5]:
ques = "040800500080760092001005470056309000009001004320500010000200700700090030005008026"
println(FindSolution(JuliaStringToArray(ques)))

947812563583764192261935478156349287879621354324587619698253741712496835435178926
